### Tratamento das imagens em diferentes camadas

In [1]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(32,32,3)),

    layers.Resizing(32,32),
    layers.Rescaling(1./255),

    layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same'),           
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),

    layers.Dense(10, activation=activations.softmax, kernel_initializer=initializers.RandomNormal())
])

### Configurando o optimizer

In [2]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

### Selecionando as imagens para o treinamento e teste

In [3]:
from tensorflow.keras import utils

path = './Cifra10'
batch_size = 64

train = utils.image_dataset_from_directory(
    directory=path + '/train',
    shuffle = True,
    seed = 1,
    subset = 'training',
    validation_split = 0.1,
    image_size = (32,32),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/test',
    shuffle = True,
    seed = 1,
    subset = 'validation',
    validation_split = 0.1,
    image_size = (32,32),
    batch_size = batch_size
)

Found 50000 files belonging to 10 classes.
Using 45000 files for training.
Found 10000 files belonging to 10 classes.
Using 1000 files for validation.


### Realizando o treinamento do modelo

In [4]:
from tensorflow.keras import callbacks

model_path = "./cifra10.keras"

patience = 5
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - loss: 1.6228 - sparse_categorical_accuracy: 0.4066 - val_loss: 1.2708 - val_sparse_categorical_accuracy: 0.5310
Epoch 2/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - loss: 1.2115 - sparse_categorical_accuracy: 0.5691 - val_loss: 1.0328 - val_sparse_categorical_accuracy: 0.6320
Epoch 3/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - loss: 1.0367 - sparse_categorical_accuracy: 0.6315 - val_loss: 0.9248 - val_sparse_categorical_accuracy: 0.6560
Epoch 4/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - loss: 0.9302 - sparse_categorical_accuracy: 0.6729 - val_loss: 0.8663 - val_sparse_categorical_accuracy: 0.6810
Epoch 5/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - loss: 0.8496 - sparse_categorical_accuracy: 0.7012 - val_loss: 0.8796 - val_sparse_categorical_accuracy: 0.6800
Epoch 6/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - loss: 0.7778 - sparse_categorical_accuracy: 0.7268 - val_loss: 0.8477 - val_sparse_categorical_acc